# The Battle of the Neighborhoods - Week 2

### Import Required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'K2XHN5JGEUJCU4BPB1YNDZ2AVNCYLUD3XPNA0DPGOIULVUFP' # your Foursquare ID
CLIENT_SECRET = 'XTV3DQUBBH0HNRXOG4HBMQCM04XSEF2LPDSLA3EJAZ0POCTZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
REDIRECT_URI = 'https://www.google.com'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2XHN5JGEUJCU4BPB1YNDZ2AVNCYLUD3XPNA0DPGOIULVUFP
CLIENT_SECRET:XTV3DQUBBH0HNRXOG4HBMQCM04XSEF2LPDSLA3EJAZ0POCTZ


### Top Tech Cities in the US
- Austin
- Boston
- Chicago
- Colorado
- Los Angeles
- NYC
- San Francisco
- Seattle

In [3]:
LIMIT = 100 # Maximum is 100
cities = ['Austin, TX', 'Boston, MA', 'Chicago, IL', 'Denver, CO', 'Los Angeles, LA', 'New York, NY', 'San Francisco, CA', 'Seattle, WA']
offices = {}
it_services = {}

for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?\
            &client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'\
        .format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city,
            10000,
            LIMIT,
            "4bf58dd8d48988d124941735"         # OFFICE CATEGORY ID
        ) 
    offices[city] = requests.get(url).json()
    
    url = 'https://api.foursquare.com/v2/venues/explore?\
            &client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'\
        .format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city,
            10000,
            LIMIT,
            "52f2ab2ebcbc57f1066b8b36"         # IT Services CATEGORY ID
        ) 
    it_services[city] = requests.get(url).json()


In [4]:
offices[cities[0]]

{'meta': {'code': 200, 'requestId': '5e933e7b0f59680026848e9f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'austin tx',
   'center': {'lat': 30.26715, 'lng': -97.74306},
   'displayString': 'Austin, TX, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 30.516862999999997,
      'lng': -97.561489},
     'sw': {'lat': 30.098658999999998, 'lng': -97.938383}}},
   'slug': 'austin-texas',
   'longId': '72057594042599590'},
  'headerLocation': 'Austin',
  'headerFullLocation': 'Austin',
  'headerLocationGranularity': 'city',
  'query': 'office',
  'totalResults': 300,
  'suggestedBounds': {'ne': {'lat': 30.320026109785523,
    'lng': -97.68409588996124},
   'sw': {'lat': 30.2062215940108, 'lng': -97.85913031081404}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'Thi

In [5]:
it_services[cities[0]]

{'meta': {'code': 200, 'requestId': '5e933f260be7b4001b41a524'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'austin tx',
   'center': {'lat': 30.26715, 'lng': -97.74306},
   'displayString': 'Austin, TX, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 30.516862999999997,
      'lng': -97.561489},
     'sw': {'lat': 30.098658999999998, 'lng': -97.938383}}},
   'slug': 'austin-texas',
   'longId': '72057594042599590'},
  'headerLocation': 'Austin',
  'headerFullLocation': 'Austin',
  'headerLocationGranularity': 'city',
  'query': 'it services',
  'totalResults': 163,
  'suggestedBounds': {'ne': {'lat': 30.400899984844024, 'lng': -97.688796825},
   'sw': {'lat': 30.147730286435994, 'lng': -97.835892675}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot 

In [6]:
df_offices={}
df_it_services={}

for city in cities:
    office_venues = pd.json_normalize(offices[city]['response']['groups'][0]['items'])
    it_services_venues = pd.json_normalize(offices[city]['response']['groups'][0]['items'])
    
    df_offices[city] = office_venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_it_services[city] = it_services_venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    
    df_offices[city].columns = ['Name', 'Address', 'Lat', 'Lng']
    df_it_services[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [7]:
df_offices[cities[0]]

,Name,Address,Lat,Lng
0,Frost Tower Financial Center,401 Congress Ave,30.266304,-97.743444
1,LatinWorks,206 E 9th St,30.270464,-97.739655
2,Cohn & Wolfe Austin,206 E 9th St Fl 16,30.270896,-97.739778
3,Silicon Labs,400 W Cesar Chavez St,30.264733,-97.748232
4,Norton Rose Fulbright,600 Congress Ave Ste 600,30.262443,-97.742881
5,Texas Motor Transport Association (TMTA),700 E 11th St,30.271078,-97.733920
6,Winstead,401 Congress Ave Ste 2100,30.266450,-97.742633
7,One American Center,600 Congress Ave,30.268527,-97.742831
8,Facebook II,600 Congress Ave,30.268598,-97.743222
9,Convey,200 E 6th St Ste 300,30.267675,-97.741148


In [8]:
df_it_services[cities[0]]

,Name,Address,Lat,Lng
0,Frost Tower Financial Center,401 Congress Ave,30.266304,-97.743444
1,LatinWorks,206 E 9th St,30.270464,-97.739655
2,Cohn & Wolfe Austin,206 E 9th St Fl 16,30.270896,-97.739778
3,Silicon Labs,400 W Cesar Chavez St,30.264733,-97.748232
4,Norton Rose Fulbright,600 Congress Ave Ste 600,30.262443,-97.742881
5,Texas Motor Transport Association (TMTA),700 E 11th St,30.271078,-97.733920
6,Winstead,401 Congress Ave Ste 2100,30.266450,-97.742633
7,One American Center,600 Congress Ave,30.268527,-97.742831
8,Facebook II,600 Congress Ave,30.268598,-97.743222
9,Convey,200 E 6th St Ste 300,30.267675,-97.741148


In [9]:
maps = {}
office_vs_services_df = {}
temp = []

geolocator = Nominatim(user_agent="office_explorer")
for city in cities:    
    address = city
    location = geolocator.geocode(address)
    city_lat = location.latitude
    city_lng = location.longitude
    # print('The geograpical coordinate of {} City are {}, {} using geopy.'.format(city, city_lat, city_lng))
    
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)

    # add office markers to map
    for lat, lng, label in zip(df_offices[city]['Lat'], df_offices[city]['Lng'], df_offices[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='#003753',  # Shade of Blue
            fill=True,
            fill_color='#005681',  # Shade of Blue
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city]) 
        
    # add markers for IT Services to map
    for lat, lng, label in zip(df_it_services[city]['Lat'], df_it_services[city]['Lng'], df_it_services[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color='b30000',  # Shade of red
            fill=True,
            fill_color='#ff4d4d',  # Shade of red
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city]) 
    
    offices_total = offices[city]['response']['totalResults']
    it_services_total = it_services[city]['response']['totalResults']
    percentage = 1-(it_services_total/offices_total)
    
    temp.append([city, offices_total, it_services_total,(percentage * 100)])
    

office_vs_services_df = pd.DataFrame(temp, columns = ['City', 'Total Offices', 'Total IT Services', 'Opportunity (%)'])

In [10]:
office_vs_services_df.sort_values(['Opportunity (%)', 'Total Offices', 'Total IT Services'], axis=0, ascending=False, inplace=True)
office_vs_services_df = office_vs_services_df.reset_index(drop=True)
office_vs_services_df

,City,Total Offices,Total IT Services,Opportunity (%)
0,"Boston, MA",279,113,59.498208
1,"Los Angeles, LA",329,161,51.063830
2,"Seattle, WA",285,147,48.421053
3,"Austin, TX",300,163,45.666667
4,"San Francisco, CA",322,185,42.546584
5,"Chicago, IL",335,194,42.089552
6,"New York, NY",375,246,34.400000
7,"Denver, CO",292,218,25.342466


In [11]:
maps[office_vs_services_df.iloc[0]['City']]

In [12]:
maps[office_vs_services_df.iloc[1]['City']]

In [13]:
maps[office_vs_services_df.iloc[2]['City']]

In [14]:
maps = {}

geolocator = Nominatim(user_agent="Mean_Plot")
for city in cities:    
    address = city
    location = geolocator.geocode(address)
    city_lat = location.latitude
    city_lng = location.longitude
    # print('The geograpical coordinate of {} City are {}, {} using geopy.'.format(city, city_lat, city_lng))
    
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)
    offices_mean_coor = [df_offices[city]['Lat'].mean(), df_offices[city]['Lng'].mean()]

    # add office markers to map
    for lat, lng, label in zip(df_offices[city]['Lat'], df_offices[city]['Lng'], df_offices[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='#003753',  # Shade of Blue
            fill=True,
            fill_color='#005681',  # Shade of Blue
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city]) 
        folium.PolyLine([offices_mean_coor, [lat, lng]], color="#3d6e3c", weight=1.5, opacity=0.5).add_to(maps[city])
                                                                                                   
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        offices_mean_coor,
        radius=10,
        popup=label,
        color='#6bc06a',
        fill=True,
        fill_color='#a7e3a6',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])
    

In [15]:
maps[office_vs_services_df.iloc[0]['City']]

In [16]:
maps[office_vs_services_df.iloc[1]['City']]

In [17]:
maps[office_vs_services_df.iloc[2]['City']]